# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
#Import cities csv as data frame
cities_file_path=os.path.join("output_data","cities.csv")
weather_df=pd.read_csv(cities_file_path)
weather_0_df=weather_df.loc[:,["City","Latitude","Longitude","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Date"]]
weather_0_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,‘Āhuimanu,21.4447,-157.8378,297.15,73,90,6.17,US,1611863517
1,Santiago,-33.4569,-70.6483,303.71,24,75,3.60,CL,1611863227
2,Rikitea,-23.1203,-134.9692,299.64,68,89,2.82,PF,1611863518
3,Upernavik,72.7868,-56.1549,248.70,88,70,4.92,GL,1611863518
4,Cape Town,-33.9258,18.4232,293.71,63,0,4.12,ZA,1611862870
...,...,...,...,...,...,...,...,...,...
579,Geraldton,-28.7667,114.6000,293.15,73,0,4.63,AU,1611863630
580,Novyy Svit,47.8059,38.0211,273.49,94,26,5.13,UA,1611863630
581,Port Hardy,50.6996,-127.4199,278.15,81,20,1.54,CA,1611863594
582,Ancud,-41.8697,-73.8203,287.15,82,75,2.57,CL,1611863631


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [35]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)


# Store latitude and longitude in locations
locations = weather_0_df[["Latitude", "Longitude"]]
# coordinates=zip(weather_df['Latitude'],weather_df['Longtitude'])

#Store Weights for Heat Map
humidity=weather_0_df["Humidity"].astype(float)

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
#Plot Heat Map
fig=gmaps.figure(layout=figure_layout)

#Create Heatmap Layer
heat_layer= gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=5)


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
#Ideal Weather Conditions
    # 70 < Max Temperature < 80 Farenheit, which is ~294 to 300 K
    # Wind Speed < 3
    # Humidity < 90
    # Cloudiness < 30

#Filter Original Data Frame to Ideal Weather Conditions
weather_filter=(weather_0_df["Max Temp"]>294)&(weather_0_df["Max Temp"]<300) &(weather_0_df["Wind Speed"]<3)&(weather_0_df["Humidity"]<90)&(weather_0_df["Cloudiness"]<30)

#New Hotel Search Data Frame
hotel_df=weather_0_df.loc[weather_filter,:]
hotel_df
# len(hotel_df)

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,Taltal,-25.4000,-70.4833,294.52,61,25,2.72,CL,1611863524
43,Āsosa,10.0667,34.5333,294.32,27,0,2.12,ET,1611863525
105,São Filipe,14.8961,-24.4956,297.36,66,0,1.93,CV,1611863536
129,Mahanoro,-19.9000,48.8000,298.90,83,0,1.89,MG,1611863541
165,Ibanda,-2.7974,32.4746,296.15,83,20,1.41,TZ,1611863548
208,Port Blair,11.6667,92.7500,299.06,83,16,2.60,IN,1611863556
218,Kuala Terengganu,5.3302,103.1408,299.15,83,20,1.03,MY,1611863558
219,Cap Malheureux,-19.9842,57.6142,298.71,79,0,1.34,MU,1611863558
223,Sambava,-14.2667,50.1667,299.39,81,18,1.60,MG,1611863559
225,Hervey Bay,-25.2986,152.8535,296.48,80,5,1.34,AU,1611863559


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
#Starter URL: https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set the parameters used in the API call
lat_long=''
distance=5000
target_type='lodging'

query_parameters={
    'key': g_key,
    'location':lat_long,
    'radius':distance,
    'type':target_type
}

#Using iterrows, iterate through Hotel Data Frame, use Latitude 
#and Longitude to find nearest hotel and store it in Data Frame 
#under Hotel Name column

# for index, each_row in hotel_df.head(5).iterrows():
for index, each_row in hotel_df.iterrows():
    
    try:
        #Get Latitude, Lontitude List
        query_parameters['location']=f"{each_row['Latitude']}, {each_row['Longitude']}"
    #     print(lat_long)

        # Compile Query URL and make API request
        response=requests.get(base_url,params=query_parameters)
        data=response.json()
#         pprint(data)

        #Get Hotel name and add it to the Hotel Data Frame
        hotel_df.loc[index,"Hotel Name"]=data['results'][0]['name']
    
    except:
        print(f"Unable to locate a hotel in {each_row['City']} within {distance} meters.")
#         print(f"Unable to locate a hotel in {each_row['City']} within {distance} meters of {query_parameters['location']} coordinates.")

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Unable to locate a hotel in Lola within 5000 meters.


In [38]:
#Drop any row where there is an NaN, find number of cities have hotel within 500 meter
hotel_1_df=hotel_df.dropna(axis=0,how='any')
len(hotel_1_df)

21

In [39]:
#Number of cities in original hotel data frame, before dropping NaN
len(hotel_df)

22

In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_1_df.iterrows()]
locations = hotel_1_df[["Latitude", "Longitude"]]

In [41]:
# Add marker layer ontop of heat map
# fig=gmaps.figure()

markers=gmaps.marker_layer(locations,info_box_content =hotel_info)
# markers=gmaps.marker_layer(locations,hover_text=hotel_info)

fig.add_layer(heat_layer)

fig.add_layer(markers)

# Display figure

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…